In [2]:
import pymysql
import json
import pandas as pd
conn = pymysql.connect(host = '120.55.167.182',user = 'root',password = 'wda20190707',database = 'dota')
cursor = conn.cursor()

sql = '''
select player_1_hero_id,player_2_hero_id,player_3_hero_id,
       player_4_hero_id,player_5_hero_id,player_6_hero_id,player_7_hero_id
    ,player_8_hero_id,player_9_hero_id,player_10_hero_id,win
from dota.`match`
where (game_mode = 22 or game_mode = 3) and 
    not (player_1_hero_id = 'None'
   or player_2_hero_id ='None'
   or player_3_hero_id = 'None'
   or player_4_hero_id ='None'
   or player_5_hero_id = 'None'
   or player_6_hero_id ='None'
   or player_7_hero_id = 'None'
   or player_8_hero_id ='None'
   or player_9_hero_id = 'None'
   or player_10_hero_id ='None')
   and skill = "3" and patch = "41"

'''
cursor.execute(sql)
match_data = cursor.fetchall()
match_data = list(match_data)
match_data = list(map(list,match_data))
match_df = pd.DataFrame(match_data)
columns = []
for i in range(1,11):
    columns.append("player_"+str(i)+"_hero_id")
columns.append("target")
match_df.columns = columns

hero_stats = pd.read_csv("Hero2Vector/input/hero_stats.csv")
print(hero_stats.columns)

kz,hx,fz,ts,bf,xs,nj,tj = {},{},{},{},{},{},{},{}
for index,row in hero_stats.iterrows():
    kz[row['id']] = row['kong_zhi']
    hx[row['id']] = row['he_xin']
    fz[row['id']] = row['fu_zhu']
    ts[row['id']] = row['tao_sheng']
    bf[row['id']] = row['bao_fa']
    xs[row['id']] = row['xian_shou']
    nj[row['id']] = row['nai_jiu']
    tj[row['id']] = row['tui_jin']

for i in range(8):
    match_df['radiant_$' + str(i)] = 0
    match_df['dire_$' + str(i)] = 0

print(match_df.head(5))



Index(['Unnamed: 0', 'hero_name', 'id', 'pro_pick', 'pro_ban', 'pro_win',
       'he_xin', 'xian_shou', 'fu_zhu', 'bao_fa', 'kong_zhi', 'nai_jiu',
       'tui_jin', 'tao_sheng'],
      dtype='object')
  player_1_hero_id player_2_hero_id player_3_hero_id player_4_hero_id  \
0               93              121                9               74   
1                9               10               19               84   
2               23                6               74                9   
3               26                6              120               33   
4               26               82              120               81   

  player_5_hero_id player_6_hero_id player_7_hero_id player_8_hero_id  \
0               26               53               23               27   
1               16               25               88                6   
2                3              106               70              102   
3               89               42               25               5

In [3]:
# 检查英雄ID为None的数据是否存在
for col in match_df.columns:
    if col.startswith("player"):
        print(col)
        print(len(match_df[match_df[col]=='None']))


player_1_hero_id
0
player_2_hero_id
0
player_3_hero_id
0
player_4_hero_id
0
player_5_hero_id
0
player_6_hero_id
0
player_7_hero_id
0
player_8_hero_id
0
player_9_hero_id
0
player_10_hero_id
0


In [4]:
# 分别计算天辉和夜宴阵容的8维能力维度总和
# 即 控制、核心、爆发、推进、辅助、先手、耐久、逃生
# （偷懒用拼音了嘻嘻）
j = 0
for dic in kz,hx,fz,ts,bf,xs,nj,tj:


        # 产生8*2列 夜宴天辉分别8列
        for i in range(5):
            #print(match_df['radiant_$'+str(j)])
            match_df['radiant_$'+str(j)] = match_df['radiant_$'+str(j)]+match_df["player_" + str(i + 1) + "_hero_id"].apply(lambda x: int(dic[int(x)]))

        for i in range(5,10):
            #print(match_df['dire_$' + str(j)])
            match_df['dire_$' + str(j)] = match_df['dire_$' + str(j)]+match_df["player_" + str(i + 1) + "_hero_id"].apply(lambda x: int(dic[int(x)]))
        j = j + 1

In [ ]:
import plotly.express as px
# ployly 绘图库

In [ ]:
# dataexplore 阶段 
# 分析观察各个维度与胜负的关系情况


# 基本信息
# match_df.columns

# Index(['player_1_hero_id', 'player_2_hero_id', 'player_3_hero_id',
#        'player_4_hero_id', 'player_5_hero_id', 'player_6_hero_id',
#        'player_7_hero_id', 'player_8_hero_id', 'player_9_hero_id',
#        'player_10_hero_id', 'target', 'radiant_kongzhi', 'dire_kongzhi',
#        'radiant_hexin', 'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu',
#        'radiant_taosheng', 'dire_taosheng', 'radiant_baofa', 'dire_baofa',
#        'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu', 'dire_naijiu',
#        'radiant_tuijin', 'dire_tuijin'],
#       dtype='object')

# 264367
# len(match_df)

In [5]:
# 修改列名为了方便
columns = ['player_1_hero_id', 'player_2_hero_id', 'player_3_hero_id',
       'player_4_hero_id', 'player_5_hero_id', 'player_6_hero_id',
       'player_7_hero_id', 'player_8_hero_id', 'player_9_hero_id',
       'player_10_hero_id', 'target', 'radiant_kongzhi', 'dire_kongzhi', 'radiant_hexin',
       'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu', 'radiant_taosheng', 'dire_taosheng',
       'radiant_baofa', 'dire_baofa', 'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu',
       'dire_naijiu', 'radiant_tuijin', 'dire_tuijin']

match_df.columns = columns

In [6]:
match_df['kongzhi_diff'] = match_df['radiant_kongzhi']-match_df['dire_kongzhi']
match_df['taosheng_diff'] = match_df['radiant_taosheng']-match_df['dire_taosheng']
match_df['hexin_diff'] = match_df['radiant_hexin']-match_df['dire_hexin']
match_df['naijiu_diff'] = match_df['radiant_naijiu']-match_df['dire_naijiu']
match_df['xianshou_diff'] = match_df['radiant_xianshou']-match_df['dire_xianshou']
match_df['baofa_diff'] = match_df['radiant_baofa']-match_df['dire_baofa']
match_df['tuijin_diff'] = match_df['radiant_tuijin']-match_df['dire_tuijin']
match_df['fuzhu_diff'] = match_df['radiant_fuzhu']-match_df['dire_fuzhu']

In [7]:
match_df.head(5)

,player_1_hero_id,player_2_hero_id,player_3_hero_id,player_4_hero_id,player_5_hero_id,player_6_hero_id,player_7_hero_id,player_8_hero_id,player_9_hero_id,player_10_hero_id,...,radiant_tuijin,dire_tuijin,kongzhi_diff,taosheng_diff,hexin_diff,naijiu_diff,xianshou_diff,baofa_diff,tuijin_diff,fuzhu_diff
0,93,121,9,74,26,53,23,27,69,6,...,0,7,1,6,-2,-3,-3,4,-7,1
1,9,10,19,84,16,25,88,6,33,112,...,2,3,-2,6,3,5,2,2,-1,0
2,23,6,74,9,3,106,70,102,120,86,...,1,0,5,-2,-1,-2,-1,4,1,-2
3,26,6,120,33,89,42,25,51,23,69,...,5,0,-1,3,2,-6,0,-4,5,2
4,26,82,120,81,101,62,18,19,21,102,...,3,2,4,1,0,-3,2,2,1,3


In [ ]:
df = match_df[['target', 'radiant_kongzhi', 'dire_kongzhi',
       'radiant_hexin', 'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu',
       'radiant_taosheng', 'dire_taosheng', 'radiant_baofa', 'dire_baofa',
       'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu', 'dire_naijiu',
       'radiant_tuijin', 'dire_tuijin', 'kongzhi_diff', 'taosheng_diff',
       'hexin_diff', 'naijiu_diff', 'xianshou_diff', 'baofa_diff',
       'tuijin_diff', 'fuzhu_diff']]

In [ ]:
df.head(5)

In [ ]:
dimension = ['kongzhi','hexin','fuzhu','baofa','taosheng','naijiu','xianshou','tuijin']

In [ ]:
df['radiant_average'] = 0
df['dire_average'] = 0
for d in dimension:
    df['radiant_average'] = df['radiant_average']+df['radiant_'+d]
    df['dire_average'] = df['dire_average']+df['dire_'+d]


In [ ]:
df.columns

In [ ]:
# 总维度值差超过5的占据总数据量一半,而根据该差值
test_df = df[(abs(df['dire_average']-df['radiant_average']))>8]

In [ ]:
# target = 0 ：18900
# target = 1 ：22000
test_df[test_df['target']=="0"]
test_df[(test_df['target']=="0") & (test_df['dire_average']-test_df['radiant_average']<-8)]

In [ ]:
test_df['kongzhi_diff'].value_counts()

In [ ]:
import sklearn

In [8]:
relation_df = pd.read_sql("select * from dota.hero_relationship",conn)

In [9]:
d = {}
for index,row in relation_df.iterrows():
    tmp_tuple = (row['hero_id'],row['target_hero_id'])
    d[tmp_tuple] = row['win_rate']


In [ ]:
# 根据论文，将天辉方相对夜魇方每个英雄的相对胜率进行求个，称为新的一列

In [10]:
def f(hero_1,hero_6,hero_7,hero_8,hero_9,hero_10):
    sum_win_rate = 0
    hero_1 = int(hero_1)
    hero_6 = int(hero_6)
    hero_7 = int(hero_7)
    hero_8 = int(hero_8)
    hero_9 = int(hero_9)
    hero_10 = int(hero_10)
    l = 0
    if((hero_1,hero_6) in d.keys()):
        l = l + d[(hero_1,hero_6)]
    if((hero_1,hero_7) in d.keys()):
        l = l + d[(hero_1,hero_7)]
    if((hero_1,hero_8) in d.keys()):
        l = l + d[(hero_1,hero_8)]
    if((hero_1,hero_9) in d.keys()):
        l = l + d[(hero_1,hero_9)]
    if((hero_1,hero_10) in d.keys()):
        l = l + d[(hero_1,hero_10)]
    return l


match_df['sum_win_rate_relative'] = match_df.apply(
    lambda x:f(x.player_1_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_2_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_3_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_4_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_5_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    ,axis = 1)



In [11]:
match_df[['sum_win_rate_relative','target']].sort_values(by = "sum_win_rate_relative")

,sum_win_rate_relative,target
21069,9.62,1
18489,9.64,0
13615,9.70,1
12736,9.89,0
38591,9.90,1
35982,9.96,1
19342,9.98,1
34125,9.98,0
31667,9.98,0
23253,9.99,0


In [14]:
match_df.columns

Index(['player_1_hero_id', 'player_2_hero_id', 'player_3_hero_id',
       'player_4_hero_id', 'player_5_hero_id', 'player_6_hero_id',
       'player_7_hero_id', 'player_8_hero_id', 'player_9_hero_id',
       'player_10_hero_id', 'target', 'radiant_kongzhi', 'dire_kongzhi',
       'radiant_hexin', 'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu',
       'radiant_taosheng', 'dire_taosheng', 'radiant_baofa', 'dire_baofa',
       'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu', 'dire_naijiu',
       'radiant_tuijin', 'dire_tuijin', 'kongzhi_diff', 'taosheng_diff',
       'hexin_diff', 'naijiu_diff', 'xianshou_diff', 'baofa_diff',
       'tuijin_diff', 'fuzhu_diff', 'sum_win_rate_relative'],
      dtype='object')

In [17]:
def one_hot(hero_1,hero_2,hero_3,hero_4,hero_5):
    v = ["0" for x in range(130)]
    v[int(hero_1)] = "1"
    v[int(hero_2)] = "1"
    v[int(hero_3)] = "1"
    v[int(hero_4)] = "1"
    v[int(hero_5)] = "1"
    return ",".join(v)

match_df['radiant_vector'] = match_df.apply(lambda x:one_hot(x.player_1_hero_id,x.player_2_hero_id,x.player_3_hero_id,x.player_4_hero_id,x.player_5_hero_id),axis = 1)
match_df['dire_vector'] = match_df.apply(lambda x:one_hot(x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id),axis = 1)

In [31]:
radiant_df = match_df['radiant_vector'].str.split(",",expand = True)
dire_df = match_df['dire_vector'].str.split(",",expand = True)

In [36]:
# 删除英雄id不存在的列，因为opendota的英雄ID不是连续的
no_hero_id =  [24,115,116,117,118,122,123,124,125,126,127,128]
for id in no_hero_id:
    del dire_df[id]
    del radiant_df[id]

c = list(dire_df.columns)

dire_df.columns = list(map(lambda x:"dire_"+str(x),c))
radiant_df.columns = list(map(lambda x:"radiant_"+str(x),c))

In [53]:
train_df = pd.concat((radiant_df,dire_df),axis = 1)

In [55]:
train_df = pd.concat((train_df,match_df['sum_win_rate_relative']),axis= 1)


In [56]:
train_df

,radiant_0,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,radiant_6,radiant_7,radiant_8,radiant_9,...,dire_110,dire_111,dire_112,dire_113,dire_114,dire_119,dire_120,dire_121,dire_129,sum_win_rate_relative
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,11.62
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,11.81
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,12.91
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,13.30
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12.25
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,11.39
6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,12.59
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13.07
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12.34
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,11.87
